# Predicting Stock Prices from Expected Earnings
## 1. Overview
This project uses a Recurrent Neural Network (RNN) through TensorFlow to predict stock prices the day after an earnings report. It takes as input the daily stock prices for the (approximately) 3-month period leading up to the earnings report, as well as the expected Earnings Per Share (EPS) prior to the report. Its output is the expected % difference between the next day's closing price and the previous day's high price.
## 2. Collecting and Cleaning Data
Data will be taken from the last 8 years' historical stock prices and earnings reports. It is housed in stocks_latest, which you should download on your own at <a href="https://www.kaggle.com/tsaustin/us-historical-stock-prices-with-earnings-data">this link</a>.

First, let's import the packages we'll be using.

In [100]:
import os
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import seaborn as sns
import tensorflow as tf
from IPython.display import display, clear_output
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Next, we'll extract our data as pandas dataframes.

In [2]:
stocks_data_path = "./datasets/stock_prices_latest.csv"
earnings_data_path = "./datasets/earnings_latest.csv"

stocks_df = pd.read_csv(stocks_data_path)
earnings_df = pd.read_csv(earnings_data_path)

Let's see what data our stocks file contains:

In [3]:
stocks_df.info()
stocks_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23528435 entries, 0 to 23528434
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   symbol             object 
 1   date               object 
 2   open               float64
 3   high               float64
 4   low                float64
 5   close              float64
 6   close_adjusted     float64
 7   volume             int64  
 8   split_coefficient  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 1.6+ GB


,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,49.7013,20032017,1.0
1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,22.5902,30977700,1.0
2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,18.0802,41591300,1.0
3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,27.2232,288121200,1.0
4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,38.6773,74640000,1.0


We'll do the following to clean up this dataset:
1. Remove split_coefficient and close, opting for close_adjusted instead
2. Rename close_adjusted to close
3. Convert our dates to datetime objects in pandas
4. Sort first by symbol, then by date

In [87]:
stocks_df.drop(['split_coefficient','close'], axis=1, errors='ignore', inplace=True)
stocks_df.rename(columns={'close_adjusted':'close'}, inplace=True)
stocks_df['date'] = pd.to_datetime(stocks_df.date)
stocks_df.sort_values(by=['symbol','date'], inplace=True)
stocks_df.head()

,symbol,date,open,high,low,volume
19762470,A,1999-11-18,45.50,50.00,40.00,44739900
19762410,A,1999-11-19,42.94,43.00,39.81,10897100
19762440,A,1999-11-22,41.31,44.00,40.06,4705200
19762399,A,1999-11-23,42.50,43.63,40.25,4274400
19762394,A,1999-11-24,40.13,41.94,40.00,3464400


We'll be feeding our NN the stock prices for each 90-day period leading up to an earnings report. First, we'll group the stock data by symbol and save as numpy arrays for faster processing.

In [88]:
stocks_sep = dict(tuple(stocks_df.groupby('symbol')))
symbol1 = stocks_sep['ZM']
symbol1.head()

,symbol,date,open,high,low,volume
20646711,ZM,2019-04-18,65.00,66.000,60.321,25764659
20646712,ZM,2019-04-22,61.00,68.900,59.940,9949738
20646710,ZM,2019-04-23,66.87,74.169,65.550,6786513
20646715,ZM,2019-04-24,71.40,71.500,63.160,4973529
20646709,ZM,2019-04-25,64.74,66.850,62.600,3863275


Now we'll need to use our earnings data, and particularly the dates of earnings periods for each company, to further separate our data into 90-day chunks. Let's clean and explore our earnings data:

In [89]:
earnings_df.dropna(inplace=True)
earnings_df.drop(['release_time', 'qtr'], axis=1, errors='ignore', inplace=True)
earnings_df['date'] = pd.to_datetime(earnings_df.date)
earnings_df.sort_values(by=['symbol', 'date'], inplace=True)
earnings_df.info()
earnings_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77282 entries, 14 to 160659
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   symbol          77282 non-null  object        
 1   date            77282 non-null  datetime64[ns]
 2   eps_est         77282 non-null  float64       
 3   eps             77282 non-null  float64       
 4   earnings_start  77282 non-null  datetime64[ns]
 5   earnings_end    77282 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(1)
memory usage: 4.1+ MB


,symbol,date,eps_est,eps,earnings_start,earnings_end
14,A,2012-11-19,0.80,0.84,2012-09-04,2012-11-16
15,A,2013-02-14,0.66,0.63,2012-11-20,2013-02-13
16,A,2013-05-14,0.67,0.77,2013-02-15,2013-05-13
17,A,2013-08-14,0.62,0.68,2013-05-15,2013-08-13
18,A,2013-11-14,0.76,0.81,2013-08-15,2013-11-13


We'll need the start and end dates for each earnings period to split our input data. Let's calculate them and assign them to two new columns in earnings_df.

In [90]:
def calculate_earnings_start(df):
    date_df = pd.DataFrame(df['date'])
    date_df = df['date'].shift(periods=1) + BDay(1)
    date_df.iat[0] = df['date'].iloc[0] - BDay(54)
    return date_df

def calculate_earnings_end(df):
    return df['date'] - BDay(1)

earnings_df['earnings_start'] = calculate_earnings_start(earnings_df)
earnings_df['earnings_end'] = calculate_earnings_end(earnings_df)
earnings_df.head()

,symbol,date,eps_est,eps,earnings_start,earnings_end
14,A,2012-11-19,0.80,0.84,2012-09-04,2012-11-16
15,A,2013-02-14,0.66,0.63,2012-11-20,2013-02-13
16,A,2013-05-14,0.67,0.77,2013-02-15,2013-05-13
17,A,2013-08-14,0.62,0.68,2013-05-15,2013-08-13
18,A,2013-11-14,0.76,0.81,2013-08-15,2013-11-13


In [91]:
earnings_sep = dict(tuple(earnings_df.groupby('symbol')))
delete_dict = []
for symbol in earnings_sep:
    try:
        stocks_sep[symbol]
    except:
        delete_dict.append(symbol)
for symbol in delete_dict:
    del earnings_sep[symbol]
symbol2 = earnings_sep['ZM']
print(symbol2)
print(len(earnings_sep))

       symbol       date  eps_est   eps earnings_start earnings_end
160389     ZM 2019-09-05    0.013  0.08     2020-03-20   2019-09-04
160390     ZM 2019-12-05    0.028  0.09     2019-09-06   2019-12-04
160391     ZM 2020-03-04    0.071  0.15     2019-12-06   2020-03-03
4328


We'll now iterate through our earnings and generate input period and output percent increases for our RNN.

_Note: this is pretty slow and could be improved with .apply() or with vectorization._

In [99]:
stocks_cnt = 0
stocks_total_cnt = len(earnings_sep)
input_data = []
output_data = []
earnings_sep_test = {k: earnings_sep[k] for k in list(earnings_sep)[:100]}
for symbol in earnings_sep_test:
    curr_stock = stocks_sep[symbol]
    stocks_cnt += 1
    if stocks_cnt % 100 == 0 or stocks_cnt == stocks_total_cnt:
        clear_output(wait=True)
        display('stocks processed: '+str(stocks_cnt)+'/'+str(stocks_total_cnt))
    for earnings_row in earnings_sep_test[symbol].itertuples():
        # compute input data
        stock_period_data = curr_stock[(curr_stock.date >= earnings_row.earnings_start) & (curr_stock.date <= earnings_row.earnings_end)]
        
        # compute output data
        earnings_prev_high = curr_stock[curr_stock.date == earnings_row.date].high.to_numpy()
        earnings_next_open = curr_stock[curr_stock.date == earnings_row.date + BDay(1)].open.to_numpy()
        percent_change = (earnings_next_open - earnings_prev_high) / earnings_prev_high
        
        # clean and store data
        stock_period_data.drop(['symbol', 'date'], axis=1, errors='ignore', inplace=True)
        input_data.append(stock_period_data)
        output_data.append(percent_change)

'stocks processed: 100/4328'

Next we'll shuffle our data and separate it into training (64%), dev (16%) and test (20%) sets.

In [152]:
x, y = shuffle(input_data, output_data)
for i in range(0, len(x)):
    x[i] = np.array(x[i].to_numpy())
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.2)

x_train = np.array(x_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

D:\code\ml\stocks_earnings\envs\lib\site-packages\ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


IndexError: tuple index out of range

In [189]:
# normalize stock data earlier on
# construct RNN
# feed input/output to neural net
# try to predict on recent earnings like ZM and CRM
# predict earnings week of 12/7

We'll now construct our neural network model with convolutional and LSTM layers.

In [143]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (2,2), padding='same', activation=tf.nn.relu,
                          input_shape=[None, None, 1]),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (2,2), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

Now we fit the model with our training data.

In [150]:
model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1)

ValueError: Error when checking input: expected conv2d_34_input to have 4 dimensions, but got array with shape (74, 4)